# CIS545: Final Project - CORD-19 Dataset
## Justin Choi
## TA: Hoyt Gong

Hi there! For this project, I opted to use the COVID-19 research challenge dataset, which they named "CORD-19" (I promise, the title wasn't a typo haha), and below you'll see the respective EDA and modeling I've done for the dataset! Hope you enjoy ~


In [3]:
import pandas as pd 
import numpy as np 
import sklearn
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'scikitlearn'